In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
lyrics = pd.read_csv(os.pardir+'/metadata/Song_Lyrics.csv')
songs = pd.read_csv(os.pardir+'/metadata/Song.csv')
stations = pd.read_csv(os.pardir+'/metadata/Station_Song.csv')
station_tags = pd.read_csv(os.pardir+'/metadata/Station_Tag.csv')
tag_info = pd.read_csv(os.pardir+'/metadata/Station_Tag_Info.csv')

#### 가사 정보에 대한 결측치 확인

In [3]:
length = lyrics['LYRICS'].apply(lambda x:len(x))
length.quantile(0.05)

175.0

In [4]:
# 가사의 길이가 100자 이하라면 가사정보가 누락된 것으로 추정한다.
lyrics[length < 100].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 4 to 1199
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   IDX      58 non-null     int64 
 1   SONG_ID  58 non-null     int64 
 2   LYRICS   58 non-null     object
dtypes: int64(2), object(1)
memory usage: 1.8+ KB


#### 각 스테이션의 구성 확인

- 57개의 스테이션, 내부에 수록된 곡의 수는 변동
- 각 스테이션마다 6개의 세부 감성으로 분류된 태깅 존재

In [5]:
stations['ST_ID'].unique().shape

(57,)

In [19]:
tags = dict()
for idx, tag in zip(tag_info['IDX'], tag_info['TAG_NAME']):
    tags[idx] = tag
print(tags)

{5322: '화나요', 5330: '행복해요', 5711: '편안해요', 5759: '슬픔', 5762: '불안', 5766: '신남'}


In [21]:
station_tags['ST_TAG'] = station_tags['ST_TAG_ID'].apply(lambda x:tags[x])

In [27]:
st_tag = dict()
for station, tag in zip(station_tags['ST_ID'], station_tags['ST_TAG']):
    st_tag[station] = tag
print(st_tag)

{363115: '행복해요', 364679: '행복해요', 454777: '행복해요', 455284: '편안해요', 456055: '화나요', 456715: '행복해요', 456828: '편안해요', 459133: '행복해요', 461177: '행복해요', 463855: '행복해요', 455966: '불안', 455981: '불안', 456091: '불안', 456109: '불안', 456133: '불안', 456141: '불안', 456143: '불안', 456427: '불안', 456657: '불안', 456757: '불안', 456206: '신남', 460784: '신남', 464453: '신남', 464556: '신남', 466118: '신남', 467316: '신남', 467742: '신남', 468779: '신남', 470388: '신남', 472220: '신남', 455052: '편안해요', 455319: '편안해요', 456422: '편안해요', 465077: '편안해요', 466008: '편안해요', 466884: '편안해요', 467710: '편안해요', 470312: '편안해요', 455995: '슬픔', 456030: '슬픔', 456107: '슬픔', 456198: '슬픔', 456333: '슬픔', 456334: '슬픔', 456375: '슬픔', 456401: '슬픔', 456402: '슬픔', 456440: '슬픔', 487508: '화나요', 488851: '화나요', 488903: '화나요', 489034: '화나요', 489238: '화나요', 493585: '화나요', 154837: '화나요', 159619: '화나요', 159735: '화나요'}


In [28]:
stations['ST_TAG'] = stations['ST_ID'].apply(lambda x:st_tag[x])

In [31]:
song_lyric = dict()
for id, lyric in zip(lyrics['SONG_ID'], lyrics['LYRICS']):
    song_lyric[id] = lyric

In [33]:
def tagging(x):
    if x not in song_lyric.keys():
        return '\n'
    else:
        return song_lyric[x]

In [34]:
stations['LYRICS'] = stations['SONG_ID'].apply(tagging)

In [37]:
titles = dict()
for id, title in zip(songs['SONG_ID'], songs['SONG_TITLE']):
    titles[id] = title 

In [38]:
stations['SONG_TITLE'] = stations['SONG_ID'].apply(lambda x:titles[x])

In [42]:
stations.to_csv(os.pardir+'/metadata/Song_Info.csv')